In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'BipedalWalker-v2'#'RoboschoolAnt-v1'#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' #'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
env = gym.make(env_name)
print(env.action_space.low)
print(env.action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1.]
[1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-05-17 17:38:20,121	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-17_17-38-20_5097/logs.
2019-05-17 17:38:20,233	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:60471 to respond...
2019-05-17 17:38:20,355	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:49001 to respond...
2019-05-17 17:38:20,356	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-05-17 17:38:20,382	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-17_17-38-20_5097/logs.
2019-05-17 17:38:20,383	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:60471',
 'object_store_address': '/tmp/ray/session_2019-05-17_17-38-20_5097/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-17_17-38-20_5097/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

a2c_config = a2c_games_configurations.bipedalwalker_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
action_space = env.action_space

agent = A2CAgent(sess,'ppo_continuous', env.observation_space, False, action_space, a2c_config)

agent.train()



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-05-17 17:38:22,063	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=5173) 
(pid=5173) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=5173) For more information, please see:
(pid=5173)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=5173)   * https://github.com/tensorflow/addons
(pid=5173) If you depend on functionality not listed there, please file an issue.
(pid=5173) 
(pid=5175) 
(pid=5175) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=5175) For more information, please see:
(pid=5175)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=5175)   * https://github.com/tensorflow/addons
(pid=5175) If you depend on functionality not listed there, please file an issue.
(pid=5175) 
(pid=5176) 
(pid=5176) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=5176) For more information, please see:
(pid=5176)   * https://github.com/tensorflow/community/blob/master/rf

(pid=5174) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=5174) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=5174) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=5174)   result = entry_point.load(False)
(pid=5228) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=5228) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=5224) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=5224) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=5224) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registrat

In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
#agent.restore('nn/a2cRoboschoolAnt-v1')

def evaluate(env,t_max=5000):
    rewards = []
    env._max_episode_steps = 5000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(t_max):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        #print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()